# Modelagem

# 1 - Bibliotecas e Dados 

In [ ]:
import pickle
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler

In [ ]:
with open('X_train.pkl', 'rb') as f:
    X_train = pickle.load(f)
with open('y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)
with open('X_test.pkl', 'rb') as f:
    X_test = pickle.load(f)
with open('y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)


## 2 - Modelos

Vou utilizarum Random Forest, a ideia é que ele é um modelo que assume não linearidade, lida bem com bases pequenas evitando um overfitting e vai poder trabalhar com os tipos de variáveis do nosso modelo
Em conjunto, vou aplicar um Gradient Boosting para ir corrigindo o gradiente do erro
Como métricas de erro vou utilizar o R² e Mean Squared Errors (MSE) para penalizar erros muito fortes

In [ ]:
# Garantir DataFrame
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

# One-hot encoding
X_train_enc = pd.get_dummies(X_train)
X_test_enc = pd.get_dummies(X_test)

X_test_enc = X_test_enc.reindex(columns=X_train_enc.columns, fill_value=0)

X_train_enc.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test_enc.replace([np.inf, -np.inf], np.nan, inplace=True)

X_train_enc.fillna(0, inplace=True)
X_test_enc.fillna(0, inplace=True)

models = {
    'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=3, min_samples_leaf=300, random_state=7),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, max_depth=2, min_samples_leaf=300, learning_rate=0.05, random_state=7)
} # Limitações para evitar overfiting

for name, model in models.items():
    model.fit(X_train_enc, y_train)
    y_pred = model.predict(X_test_enc)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"{name}: MSE = {mse:.4f}, R2 = {r2:.4f}")

Random Forest: MSE = 0.0661, R2 = -0.0044
Gradient Boosting: MSE = 0.0139, R2 = 0.7890


Com isso temos que o GB é um modelo de classificação adequado, seu erro médio é baixo e o R² elevado, mas nao ao ponto de indicar overfitting, apontam um sucesso do modelo 

### 3 - Classificando o filme The Shawshank Redemption

In [68]:
novo_filme = {
    'Series_Title': ['The Shawshank Redemption'],
    'Released_Year': [1994],
    'Certificate': ['A'],
    'Runtime': ['142 min'],
    'Genre': ['Drama'],
    'Overview': ['Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.'],
    'Meta_score': [80.0],
    'Director': ['Frank Darabont'],
    'Star1': ['Tim Robbins'],
    'Star2': ['Morgan Freeman'],
    'Star3': ['Bob Gunton'],
    'Star4': ['William Sadler'],
    'No_of_Votes': [2343110],
    'Gross': ['28,341,469'] 
}

novo_df = pd.DataFrame(novo_filme)

novo_df['Gross'] = novo_df['Gross'].str.replace(',', '').astype(float)
novo_df['Runtime'] = novo_df['Runtime'].str.replace(' min','').astype(float)

# One-hot encoding
novo_df_enc = pd.get_dummies(novo_df)

novo_df_enc = novo_df_enc.reindex(columns=X_train_enc.columns, fill_value=0)

gbm_pred = models['Gradient Boosting'].predict(novo_df_enc)
print("Gradient Boosting prevê nota:", gbm_pred[0])

Gradient Boosting prevê nota: 8.222888558454644


Considerando que The Shawshank Redemption tem nota 9.3 no IMDB, o erro foi de aproximadamente 1 ponto